In [1]:
import pandas as pd
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
import bokeh.transform as btr
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
from datetime import datetime 
import timeit

In [2]:
data = pd.read_csv("http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip", compression = "zip", encoding = "latin1")

In [3]:
pio.templates.default = "plotly_white"
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly_white'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [4]:
data 

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-29,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,MÃ©xico,97,97
1,2021-01-29,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,MÃ©xico,97,1
2,2021-01-29,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,MÃ©xico,97,97
3,2021-01-29,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,MÃ©xico,97,97
4,2021-01-29,zz8e77,1,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,MÃ©xico,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4642534,2021-01-29,7730e2,2,9,21,1,15,21,114,1,...,2,1,3,2,97,6,99,MÃ©xico,97,97
4642535,2021-01-29,740133,1,4,1,2,1,1,11,1,...,2,2,97,2,97,6,99,MÃ©xico,97,97
4642536,2021-01-29,834896,2,4,11,2,11,11,20,1,...,1,2,97,1,2,7,99,MÃ©xico,97,97
4642537,2021-01-29,5bd476,1,5,16,2,16,16,9,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97


In [5]:
last_update = data["FECHA_ACTUALIZACION"].iloc[0]
date_last_update = datetime.strptime(last_update, '%Y-%m-%d')

if date_last_update.day < 10:
    day = "0" + str(date_last_update.day) 
else:
    day = str(date_last_update.day) 

if date_last_update.month < 10:
    month = "0" + str(date_last_update.month)
else:
    month = str(date_last_update.month)

str_last_update = day + "/" + month + "/" + str(date_last_update.year) 
str_last_update

'29/01/2021'

In [6]:
data[["FECHA_ACTUALIZACION", "FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF"]] #COLUMNA QUE ME INTERESA DataFrame de una columna -----> Serie

,FECHA_ACTUALIZACION,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF
0,2021-01-29,2020-10-16,2020-10-16,9999-99-99
1,2021-01-29,2020-07-17,2020-07-20,2020-07-21
2,2021-01-29,2021-01-05,2021-01-05,9999-99-99
3,2021-01-29,2020-10-15,2020-10-15,9999-99-99
4,2021-01-29,2020-04-10,2020-04-16,9999-99-99
...,...,...,...,...
4642534,2021-01-29,2021-01-29,2021-01-29,9999-99-99
4642535,2021-01-29,2021-01-19,2021-01-29,9999-99-99
4642536,2021-01-29,2021-01-07,2021-01-13,9999-99-99
4642537,2021-01-29,2021-01-27,2021-01-29,9999-99-99


In [7]:
data[['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN']]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN
0,2021-01-29,z482b8,1
1,2021-01-29,z49a69,1
2,2021-01-29,z23d9d,1
3,2021-01-29,z24953,1
4,2021-01-29,zz8e77,1
...,...,...,...
4642534,2021-01-29,7730e2,2
4642535,2021-01-29,740133,1
4642536,2021-01-29,834896,2
4642537,2021-01-29,5bd476,1


In [8]:
data.loc[10] #INFORMACION DE LA FILA 10

FECHA_ACTUALIZACION      2021-01-29
ID_REGISTRO                  z54974
ORIGEN                            1
SECTOR                           12
ENTIDAD_UM                        9
SEXO                              2
ENTIDAD_NAC                       9
ENTIDAD_RES                       9
MUNICIPIO_RES                    12
TIPO_PACIENTE                     1
FECHA_INGRESO            2020-10-10
FECHA_SINTOMAS           2020-10-01
FECHA_DEF                9999-99-99
INTUBADO                         97
NEUMONIA                          2
EDAD                             12
NACIONALIDAD                      1
EMBARAZO                         97
HABLA_LENGUA_INDIG                2
INDIGENA                          2
DIABETES                          2
EPOC                              2
ASMA                              2
INMUSUPR                          2
HIPERTENSION                      2
OTRA_COM                          2
CARDIOVASCULAR                    2
OBESIDAD                    

In [9]:
data.loc[[1,3,6]]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
1,2021-01-29,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,MÃ©xico,97,1
3,2021-01-29,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,MÃ©xico,97,97
6,2021-01-29,z2d0c4,1,12,9,1,9,9,6,1,...,2,1,2,2,97,7,99,MÃ©xico,97,97


In [10]:
#Casos negativos, sospechosos y activos
data["CLASIFICACION_FINAL"].value_counts()

7    2365303
3    1751018
6     364324
1      87559
5      60604
4      10415
2       3316
Name: CLASIFICACION_FINAL, dtype: int64

In [11]:
#Casos negativos ---> 7
condicion_negativos = data["CLASIFICACION_FINAL"] == 7

In [12]:
condicion_negativos.value_counts()

True     2365303
False    2277236
Name: CLASIFICACION_FINAL, dtype: int64

In [13]:
 negativos = data[condicion_negativos] #Filtrando el data FRAME para casos negativos
 print("Casos Negativos =",len(negativos))

Casos Negativos = 2365303


In [14]:
# 1,2,3 CASOS CONFIRMADOS
confirmados = data[data["CLASIFICACION_FINAL"].isin([1,2,3])]
print("Casos confirmados =",len(confirmados))

Casos confirmados = 1841893


In [15]:
confirmados["CLASIFICACION_FINAL"].value_counts()

3    1751018
1      87559
2       3316
Name: CLASIFICACION_FINAL, dtype: int64

In [16]:
#CASOS SORPECHOSOS 6 --> sospechosos sin muestra", "sospechosos con posibilidad de resultado" y "sospechosos sin posibilidad de resultado".
sospechosos = data[data["CLASIFICACION_FINAL"] == 6]
print("Sospechosos Totales = ", len(sospechosos))

Sospechosos Totales =  364324


In [17]:
sospechosos_sin_muestra=data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==97)]
#sospechosos_sin_muestra
print("Sospechosos sin muestra =",len(sospechosos_sin_muestra))

Sospechosos sin muestra = 246352


In [18]:
sospechosos_con_posibilidad =data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==3)]
#sospechosos_sin_muestra
print("Sospechosos con posibilidad =",len(sospechosos_con_posibilidad))

Sospechosos con posibilidad = 35944


In [19]:
sospechosos_sin_posibilidad=data[(data["CLASIFICACION_FINAL"].isin([6,5,4]))&(data["RESULTADO_LAB"].isin([4]))]
print("Sospechosos sin posibilidad =", len(sospechosos_sin_posibilidad))

Sospechosos sin posibilidad = 153047


In [20]:
#Buscando defunciones 
data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN', 'UCI'],
      dtype='object')

In [21]:
#data["FECHA_DEF"].value_counts()

In [22]:
defunciones = confirmados[confirmados["FECHA_DEF"] != "9999-99-99"]
print("Defunciones =", len(defunciones))


Defunciones = 156579


In [23]:
grupos = confirmados.groupby("FECHA_INGRESO")  #agrupando por fecha
confirmados_por_fecha = grupos.size().to_frame("confirmados").reset_index()   #INICIALMENTE grupos.size() es una serie con la fecha como ind
confirmados_por_fecha

,FECHA_INGRESO,confirmados
0,2020-01-20,1
1,2020-01-21,1
2,2020-01-28,1
3,2020-02-19,1
4,2020-02-20,1
...,...,...
338,2021-01-25,14258
339,2021-01-26,12822
340,2021-01-27,9459
341,2021-01-28,6141


In [24]:
confirmados_por_fecha.dtypes

FECHA_INGRESO    object
confirmados       int64
dtype: object

In [25]:
confirmados_por_fecha["FECHA_INGRESO"][:3]

0    2020-01-20
1    2020-01-21
2    2020-01-28
Name: FECHA_INGRESO, dtype: object

In [26]:
confirmados_por_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")
confirmados_por_fecha

,FECHA_INGRESO,confirmados,fecha_ingreso
0,2020-01-20,1,2020-01-20
1,2020-01-21,1,2020-01-21
2,2020-01-28,1,2020-01-28
3,2020-02-19,1,2020-02-19
4,2020-02-20,1,2020-02-20
...,...,...,...
338,2021-01-25,14258,2021-01-25
339,2021-01-26,12822,2021-01-26
340,2021-01-27,9459,2021-01-27
341,2021-01-28,6141,2021-01-28


In [27]:
confirmados_por_fecha.dtypes

FECHA_INGRESO            object
confirmados               int64
fecha_ingreso    datetime64[ns]
dtype: object

In [28]:
defunciones_por_fecha = defunciones.groupby("FECHA_DEF").size().to_frame("defunciones").reset_index()
defunciones_por_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_fecha


,FECHA_DEF,defunciones,fecha_defuncion
0,2020-01-24,1,2020-01-24
1,2020-03-03,1,2020-03-03
2,2020-03-13,1,2020-03-13
3,2020-03-17,1,2020-03-17
4,2020-03-18,2,2020-03-18
...,...,...,...
315,2021-01-25,929,2021-01-25
316,2021-01-26,816,2021-01-26
317,2021-01-27,603,2021-01-27
318,2021-01-28,192,2021-01-28


In [29]:
sospechosos_por_fecha = sospechosos.groupby("FECHA_SINTOMAS").size().to_frame("sospechosos").reset_index()
sospechosos_por_fecha["fecha_sintomas"] = pd.to_datetime(sospechosos_por_fecha["FECHA_SINTOMAS"], format = "%Y-%m-%d")


In [30]:
bpl.output_notebook()

Loading BokehJS ...

In [31]:
# Ploting each line in different figures ...
def plotCases(abscisa, ordenada, x_title, y_title, g_title, paint ):
    fig_1 = px.line( x = abscisa, y = ordenada, title = g_title, width = 1500, height = 600) # labels parameter can be added with a dict

    fig_1.update_traces(marker = dict(color = paint), line = dict(color = paint, width = 2))
    fig_1.update_xaxes(showgrid = True, title = x_title)
    fig_1.update_yaxes(showgrid = True, title = y_title)
    fig_1.show()
    
plotCases(confirmados_por_fecha["fecha_ingreso"], confirmados_por_fecha["confirmados"], 
        "Fecha de registro", "Casos confirmados", "Casos de COVID-19 en México","#b4004e")
plotCases(defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones"], 
        "Fecha de defunción", "Defunciones", "Defunciones por COVID-19 en México" , "black")
# plotCases(sospechosos_por_fecha["fecha_sintomas"],sospechosos_por_fecha["sospechosos"], 
#         "Fecha del inicio de sintomas", "Casos sospechosos", "Casos sospechosos de COVID-19 en México", "#c67c00")


In [32]:
# Computing the comulative cases ...
def commulative_cases(data):
    data_vec = np.array(data)
    data_acumulados = np.cumsum(data_vec)    
    return pd.Series(data_acumulados)                           # Numpy array to pandas Serie

# Commulative confirmed cases ....
confirmados_por_fecha["casos_acumulados"] = commulative_cases(confirmados_por_fecha["confirmados"])
# Commulative Deaths  ....
defunciones_por_fecha["defunciones_acumuladas"] = commulative_cases(defunciones_por_fecha["defunciones"])

In [33]:
#Computing the average with the actual day plus the 6 previous days
def average_of_seven_days(serie , threshold = 25000):
    data_vect = np.array(serie)
    average = np.zeros(data_vect.size)
    counter = 0
    init = 0
    # ....
    index = np.nonzero(data_vect > threshold)
    data_vect[index] = 0
    #Computing the average
    for i in range(data_vect.size):
        if i >= 6:
            average[counter] = np.mean(data_vect[init:i+1])
            init += 1
        else:
            average[counter] = 1/7 * np.sum(data_vect[0:i+1])

        counter += 1

    return  pd.Series(np.round(average))   #Return a pandas Serie with rounded values

In [34]:
# confirmados_por_fecha["casos_promedio"] =  average_of_seven_days(confirmados_por_fecha["confirmados"])
# defunciones_por_fecha["defunciones_promedio"] = average_of_seven_days(defunciones_por_fecha["defunciones"])

In [35]:

# fig_bar1_conf = px.bar(x =confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
    
#                     labels= dict(x = "Fecha de registro", y = "Nuevos casos confirmados"), title="Nuevos casos de COVID-19 reportados diariamente", width = 900)
#fig_bar1_conf.update_traces( marker = dict(color = "#ff1744", opacity = 0.6))
# fig_bar2_def = px.bar(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"],
#                         labels = dict(x = "Fecha de defunción", y = "Defunciones"), title = "Nuevas defunciones por COVID-19 reportadas diariamente", width = 900)

# fig_bar2_def.update_traces(marker = dict(color = "black"))

# Bar plots for confirmed and death cases ...
def show_confirmed_cases(x_vect,y_vect, region, anomalia):

    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900

    fig_bar1 = go.Figure()
    fig_bar1.add_trace(go.Bar(x = x_vect, y = y_vect, 
                                marker = dict(color = "#ff1744", opacity = 0.6 ), name = "Nuevos casos"))
    #Average of seven days curve ...                             
    fig_bar1.add_trace(go.Scatter(x = x_vect, y =  average_of_seven_days(y_vect),
                                        mode = "lines", line = dict(color = "#b71c1c", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar1.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar1.update_yaxes(title = "Nuevos casos confirmados")
    fig_bar1.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevos casos de COVID-19 reportados diariamente en " + region)
    #Showing figure ... 
    fig_bar1.show()

def show_deaths(x_vect,y_vect,region, anomalia):
    
    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900
    # Death cases ...
    fig_bar2 = go.Figure()
    fig_bar2.add_trace(go.Bar(x = x_vect, y = y_vect,
                                marker = dict(color = "#546e7a", opacity = 0.75), name = "Nuevas defunciones"))
    #Average of seven days curve ...                         
    fig_bar2.add_trace(go.Scatter(x = x_vect, y =average_of_seven_days(y_vect, threshold = 2500),
                                        mode = "lines", line = dict(color = "black", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar2.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar2.update_yaxes(title = "Nuevas defunciones confirmadas")
    fig_bar2.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevas defunciones de COVID-19 reportadas diariamente en " + region)
    #Showing figure ...
    fig_bar2.show()

show_confirmed_cases(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["confirmados"], "México", False)
show_deaths(defunciones_por_fecha["fecha_defuncion"],defunciones_por_fecha["defunciones"],"México", False)

In [36]:
# Ploting the commulative confirmed and deaths cases ...

def acum_plot(x_conf_data, y_conf_data, x_def_data, y_def_data, region ):
    fig_acum = go.Figure()

    fig_acum.add_trace(go.Scatter(x = x_conf_data, y = y_conf_data,
                    mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

    fig_acum.add_trace(go.Scatter(x = x_def_data, y = y_def_data,
                    mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))

    fig_acum.update_xaxes(title = "Fecha de registro")
    fig_acum.update_yaxes(title = "Número de casos acumulados")
    fig_acum.update_layout(title = "Casos acumulados en " + region )
    fig_acum.show()


# fig_acum = go.Figure()

# fig_acum.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["casos_acumulados"],
#                     mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

# fig_acum.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones_acumuladas"],
#                     mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))



# fig_acum.update_xaxes(title = "Fecha de registro")
# fig_acum.update_yaxes(title = "Número de casos acumulados")
# fig_acum.update_layout(title = "Casos acumulados en México")
# fig_acum.show()

acum_plot(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["casos_acumulados"],
            defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones_acumuladas"], region = "México" )


In [37]:

# Ploting each line in the same figure ...

# fig_2 = px.line(confirmados_por_fecha, x = "fecha_sintomas", y = "confirmados", width = 1000, height= 300,
#                  labels= dict(fecha_sintomas = "Fecha", confirmados = "Número de personas", name = "Casos confirmados"))

fig_2 = go.Figure()

fig_2.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                mode = "lines",name = "Casos confirmados", line = dict(color = "#b4004e")) )
fig_2.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"], 
                mode = "lines",name = "Defunciones", line = dict(color = "black") ) )
fig_2.add_trace(go.Scatter(x = sospechosos_por_fecha["fecha_sintomas"], y = sospechosos_por_fecha["sospechosos"],
                mode = "lines", name = "Casos sospechosos", line = dict(color = "#c67c00")))

fig_2.update_xaxes(showgrid = True, title = "Fecha")
fig_2.update_yaxes(showgrid = True , title = "Número de personas")
fig_2.update_layout( width = 1200, height= 400 , title = "COVID-19 México")
fig_2.show()

In [38]:
# Distribution of cases per state ...................................
confirmados

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-01-29,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,MÃ©xico,97,97
1,2021-01-29,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,MÃ©xico,97,1
8,2021-01-29,z33a15,2,12,12,2,12,12,29,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
12,2021-01-29,zzcda2,1,12,2,1,2,2,4,1,...,1,2,97,2,97,1,99,MÃ©xico,97,97
14,2021-01-29,z4f06b,1,6,22,2,15,22,14,1,...,2,1,1,2,97,3,99,MÃ©xico,97,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4642517,2021-01-29,b9b189,2,4,23,2,23,23,4,2,...,2,2,97,1,1,3,99,MÃ©xico,97,2
4642520,2021-01-29,a3d735,2,4,9,1,9,9,2,1,...,2,2,97,1,1,3,99,MÃ©xico,97,97
4642529,2021-01-29,947a55,2,12,9,2,9,9,7,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97
4642537,2021-01-29,5bd476,1,5,16,2,16,16,9,1,...,1,2,97,1,1,3,99,MÃ©xico,97,97


In [39]:
def time_usage(func):
    def wrapper(*args, **kwargs):
        beg_ts = time.time()
        retval = func(*args, **kwargs)
        end_ts = time.time()
        print("elapsed time: %f" % (end_ts - beg_ts))
        return retval
    return wrapper

In [40]:
#Grouping by state ENTIDAD_RES
confirmados_por_estado = confirmados.groupby("ENTIDAD_RES").size().to_frame("Casos confirmados").reset_index()
confirmados_por_estado

,ENTIDAD_RES,Casos confirmados
0,1,20873
1,2,41983
2,3,23134
3,4,7918
4,5,59862
5,6,9356
6,7,9279
7,8,40550
8,9,474355
9,10,28669


In [41]:
# Cargando el diccionario de estados
dics = pd.read_excel("https://github.com/Riloro/mexico_covid19_dashboard/blob/master/diccionarios/201128%20Catalogos_.xlsx?raw=true", sheet_name = None)
dics.keys()

dict_keys(['Catálogo ORIGEN', 'Catálogo SECTOR', 'Catálogo SEXO', 'Catálogo TIPO_PACIENTE', 'Catálogo SI_NO', 'Catálogo NACIONALIDAD', 'Catálogo RESULTADO_LAB', 'Catálogo RESULTADO_ANTIGENO', 'Catálogo CLASIFICACION_FINAL', 'Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

In [42]:
dic_estados = dics["Catálogo de ENTIDADES"]
dic_estados

,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,AS
1,2,BAJA CALIFORNIA,BC
2,3,BAJA CALIFORNIA SUR,BS
3,4,CAMPECHE,CC
4,5,COAHUILA DE ZARAGOZA,CL
5,6,COLIMA,CM
6,7,CHIAPAS,CS
7,8,CHIHUAHUA,CH
8,9,CIUDAD DE MÉXICO,DF
9,10,DURANGO,DG


In [43]:
confirmados_por_entidad = pd.merge(confirmados_por_estado, dic_estados, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD" ) # left outer join
confirmados_por_entidad

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,20873,1,AGUASCALIENTES,AS
1,2,41983,2,BAJA CALIFORNIA,BC
2,3,23134,3,BAJA CALIFORNIA SUR,BS
3,4,7918,4,CAMPECHE,CC
4,5,59862,5,COAHUILA DE ZARAGOZA,CL
5,6,9356,6,COLIMA,CM
6,7,9279,7,CHIAPAS,CS
7,8,40550,8,CHIHUAHUA,CH
8,9,474355,9,CIUDAD DE MÉXICO,DF
9,10,28669,10,DURANGO,DG


In [44]:
confirmados_por_entidad.dtypes

ENTIDAD_RES            int64
Casos confirmados      int64
CLAVE_ENTIDAD          int64
ENTIDAD_FEDERATIVA    object
ABREVIATURA           object
dtype: object

In [45]:
# Bar plot of the confirmed cases by state
confirmados_por_entidad.sort_values("Casos confirmados", ascending = False, inplace = True ) # Ordenando por número de casos de forma descendente 
fig_bar_edo = px.bar( x = confirmados_por_entidad["ENTIDAD_FEDERATIVA"], y = confirmados_por_entidad["Casos confirmados"],
                        labels = dict(x = "Entidad Federativa", y = "Número de casos confirmados"), title = "Número de casos confirmados por estado")


fig_bar_edo.update_xaxes(type='category', tickfont=dict(size = 10))
fig_bar_edo.update_traces(marker = dict(color = "#b71c1c"), opacity = 0.8)
fig_bar_edo.show()

In [46]:
stime = timeit.default_timer()
poblaciones = pd.read_csv("http://www.conapo.gob.mx/work/models/CONAPO/Datos_Abiertos/Proyecciones2018/pob_mit_proyecciones.csv", encoding="latin1")
e = timeit.default_timer() - stime
poblaciones
print(e)
poblacion_2020 = poblaciones[poblaciones["AÑO"] == 2020]

11.084413499999997


In [47]:
grupos_pob = poblacion_2020.groupby(["CVE_GEO","ENTIDAD"])
poblacion_tot_estado = grupos_pob["POBLACION"].sum().to_frame("POBLACION").reset_index()   # Sum operation in the POBLACION column of each group

In [48]:
datos_por_estado = pd.merge(confirmados_por_entidad, poblacion_tot_estado, how = "left", left_on = "CLAVE_ENTIDAD", right_on = "CVE_GEO" )
datos_por_estado.sort_values("CVE_GEO", ascending = True, inplace = True)
#Datos por estado a CSV .....
datos_por_estado.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/poblaciones_casos_por_estado.csv",
                                            encoding="latin1", index=False)
datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION
24,1,20873,1,AGUASCALIENTES,AS,1,Aguascalientes,1434635
13,2,41983,2,BAJA CALIFORNIA,BC,2,Baja California,3634868
23,3,23134,3,BAJA CALIFORNIA SUR,BS,3,Baja California Sur,804708
31,4,7918,4,CAMPECHE,CC,4,Campeche,1000617
7,5,59862,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720
29,6,9356,6,COLIMA,CM,6,Colima,785153
30,7,9279,7,CHIAPAS,CS,7,Chiapas,5730367
14,8,40550,8,CHIHUAHUA,CH,8,Chihuahua,3801487
0,9,474355,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645
21,10,28669,10,DURANGO,DG,10,Durango,1868996


In [49]:
poblacion_tot_estado

,CVE_GEO,ENTIDAD,POBLACION
0,0,República Mexicana,127792286
1,1,Aguascalientes,1434635
2,2,Baja California,3634868
3,3,Baja California Sur,804708
4,4,Campeche,1000617
5,5,Coahuila,3218720
6,6,Colima,785153
7,7,Chiapas,5730367
8,8,Chihuahua,3801487
9,9,Ciudad de México,9018645


In [50]:
lista_estados = list(datos_por_estado["ENTIDAD"])
print(lista_estados)

['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de México', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'México', 'Michoacán', 'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro', 'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatán', 'Zacatecas']


In [51]:

# Rate of confirmed case by 100,000 people 
datos_por_estado["TASA DE CASOS POR 100K"] = 100000 * datos_por_estado["Casos confirmados"]/datos_por_estado["POBLACION"]
datos_por_estado.sort_values("TASA DE CASOS POR 100K", ascending = False, inplace = True)
datos_por_estado.reset_index(drop = True, inplace = True)

datos_por_estado

,ENTIDAD_RES,Casos confirmados,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA,CVE_GEO,ENTIDAD,POBLACION,TASA DE CASOS POR 100K
0,9,474355,9,CIUDAD DE MÉXICO,DF,9,Ciudad de México,9018645,5259.714735
1,3,23134,3,BAJA CALIFORNIA SUR,BS,3,Baja California Sur,804708,2874.831616
2,22,46622,22,QUERÉTARO,QT,22,Querétaro,2279637,2045.150171
3,27,52269,27,TABASCO,TC,27,Tabasco,2572287,2032.004982
4,26,62177,26,SONORA,SR,26,Sonora,3074745,2022.183953
5,19,105584,19,NUEVO LEÓN,NL,19,Nuevo León,5610153,1882.016408
6,5,59862,5,COAHUILA DE ZARAGOZA,CL,5,Coahuila,3218720,1859.807625
7,24,49752,24,SAN LUIS POTOSÍ,SP,24,San Luis Potosí,2866142,1735.852585
8,11,106371,11,GUANAJUATO,GT,11,Guanajuato,6228175,1707.899987
9,10,28669,10,DURANGO,DG,10,Durango,1868996,1533.925166


In [52]:
# Bar plot of the rate of confirmed cases per 100k persons 
fig_bar_cases_rate = px.bar(x = datos_por_estado["ENTIDAD"], y = datos_por_estado["TASA DE CASOS POR 100K"],
                            labels = dict(y = "Casos confirmados / 100k personas" , x = "") ,title = "Tasa de casos confirmados por cada 100,000 personas")


fig_bar_cases_rate.update_traces(marker = dict(color = "#ff616f"))
#fig_bar_cases_rate.update_xaxes(tickfont = dict(size = 11))
fig_bar_cases_rate.show()

In [53]:
# Grouping by state ENTIDAD_RES and FECHA_INGRESO
confirmados_por_estado_fecha = confirmados.groupby(["ENTIDAD_RES","FECHA_INGRESO"]).size().to_frame("confirmados").reset_index()
#Confirmados por estado a CSV .....
confirmados_por_estado_fecha.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/confirmados_por_estado.csv", encoding="latin1",                                                   index=False)
confirmados_por_estado_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_estado_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")


confirmados_por_estado_fecha

,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso
0,1,2020-02-20,1,2020-02-20
1,1,2020-03-14,1,2020-03-14
2,1,2020-03-17,1,2020-03-17
3,1,2020-03-18,3,2020-03-18
4,1,2020-03-19,1,2020-03-19
...,...,...,...,...
10117,32,2021-01-24,39,2021-01-24
10118,32,2021-01-25,153,2021-01-25
10119,32,2021-01-26,184,2021-01-26
10120,32,2021-01-27,131,2021-01-27


In [54]:
defunciones_por_estado_fecha = defunciones.groupby(["ENTIDAD_RES", "FECHA_DEF"]).size().to_frame("defunciones").reset_index()
#Defunciones por estado a CSV ............................
defunciones_por_estado_fecha.to_csv("C:/Users/user/Documents/Python Scripts/dataAnalysis/covid19_dashboard/processed_data/defunciones_por_estado.csv",
                                            encoding="latin1", index=False)
defunciones_por_estado_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_estado_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_estado_fecha

,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion
0,1,2020-04-10,1,2020-04-10
1,1,2020-04-13,1,2020-04-13
2,1,2020-04-17,1,2020-04-17
3,1,2020-04-19,1,2020-04-19
4,1,2020-04-22,1,2020-04-22
...,...,...,...,...
9052,32,2021-01-24,14,2021-01-24
9053,32,2021-01-25,11,2021-01-25
9054,32,2021-01-26,4,2021-01-26
9055,32,2021-01-27,2,2021-01-27


In [55]:
entidad_clave = datos_por_estado["ENTIDAD"].to_frame()
entidad_clave["CLAVE_ENTIDAD"] = datos_por_estado["CLAVE_ENTIDAD"]
# Merge between entidad_cave and confirmados_por_estado_fecha
confirmados_por_estado_fecha_clave = pd.merge(confirmados_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")

In [56]:
# Merge betweem entidad_clave and defunciones_por_estado_fecha
defunciones_por_estado_fecha_clave = pd.merge(defunciones_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")

In [57]:
clave_estado = 32
confirmados_mich = confirmados_por_estado_fecha_clave[confirmados_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado]
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())
confirmados_mich.reset_index(drop = True, inplace = True)
confirmados_mich["confirmados_acumulados"] = commulative_cases(confirmados_mich["confirmados"]) # Accumulated confirmed cases in Mich.
confirmados_mich

Total de casos confirmados =  24931


,ENTIDAD_RES,FECHA_INGRESO,confirmados,fecha_ingreso,ENTIDAD,CLAVE_ENTIDAD,confirmados_acumulados
0,32,2020-02-19,1,2020-02-19,Zacatecas,32,1
1,32,2020-03-03,1,2020-03-03,Zacatecas,32,2
2,32,2020-03-04,3,2020-03-04,Zacatecas,32,5
3,32,2020-03-11,1,2020-03-11,Zacatecas,32,6
4,32,2020-03-16,1,2020-03-16,Zacatecas,32,7
...,...,...,...,...,...,...,...
305,32,2021-01-24,39,2021-01-24,Zacatecas,32,24407
306,32,2021-01-25,153,2021-01-25,Zacatecas,32,24560
307,32,2021-01-26,184,2021-01-26,Zacatecas,32,24744
308,32,2021-01-27,131,2021-01-27,Zacatecas,32,24875


In [58]:
defunciones_mich = defunciones_por_estado_fecha_clave[defunciones_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado ]
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
defunciones_mich.reset_index(drop = True, inplace = True)     
defunciones_mich["defunciones_acumuladas"] = commulative_cases(defunciones_mich["defunciones"])
defunciones_mich

Total de defunciones acumuladas =  2069


,ENTIDAD_RES,FECHA_DEF,defunciones,fecha_defuncion,ENTIDAD,CLAVE_ENTIDAD,defunciones_acumuladas
0,32,2020-03-31,1,2020-03-31,Zacatecas,32,1
1,32,2020-04-04,1,2020-04-04,Zacatecas,32,2
2,32,2020-04-19,1,2020-04-19,Zacatecas,32,3
3,32,2020-04-23,1,2020-04-23,Zacatecas,32,4
4,32,2020-04-26,1,2020-04-26,Zacatecas,32,5
...,...,...,...,...,...,...,...
266,32,2021-01-24,14,2021-01-24,Zacatecas,32,2051
267,32,2021-01-25,11,2021-01-25,Zacatecas,32,2062
268,32,2021-01-26,4,2021-01-26,Zacatecas,32,2066
269,32,2021-01-27,2,2021-01-27,Zacatecas,32,2068


In [59]:
#
acum_plot(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados_acumulados"],
            defunciones_mich["fecha_defuncion"], defunciones_mich["defunciones_acumuladas"], region = confirmados_mich["ENTIDAD"].iloc[0] )
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())

Total de defunciones acumuladas =  2069
Total de casos confirmados =  24931


In [60]:
show_confirmed_cases(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)
show_deaths(defunciones_mich["fecha_defuncion"],defunciones_mich["defunciones"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)

In [61]:
jhu_daily_reports = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-17-2021.csv", encoding="utf-8")
mexico_daily_report = jhu_daily_reports[jhu_daily_reports["Country_Region"] == "Mexico"]
mexico_daily_report

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
387,NaN,NaN,Aguascalientes,Mexico,2021-01-18 05:22:34,21.8853,-102.2916,18501,1498,0,17003.0,"Aguascalientes, Mexico",1289.596308,8.096860
388,NaN,NaN,Baja California,Mexico,2021-01-18 05:22:34,30.8406,-115.2838,38697,6289,0,32408.0,"Baja California, Mexico",1064.605372,16.251906
389,NaN,NaN,Baja California Sur,Mexico,2021-01-18 05:22:34,26.0444,-111.6661,20741,835,0,19906.0,"Baja California Sur, Mexico",2577.456668,4.025843
390,NaN,NaN,Campeche,Mexico,2021-01-18 05:22:34,19.8301,-90.5349,7552,989,0,6563.0,"Campeche, Mexico",754.734329,13.095869
391,NaN,NaN,Chiapas,Mexico,2021-01-18 05:22:34,16.7569,-93.1292,8766,1259,0,7507.0,"Chiapas, Mexico",152.974495,14.362309
392,NaN,NaN,Chihuahua,Mexico,2021-01-18 05:22:34,28.6330,-106.0691,38326,4653,0,33673.0,"Chihuahua, Mexico",1008.184429,12.140583
393,NaN,NaN,Ciudad de Mexico,Mexico,2021-01-18 05:22:34,19.4326,-99.1332,407254,19417,0,387837.0,"Ciudad de Mexico, Mexico",4515.689441,4.767786
394,NaN,NaN,Coahuila,Mexico,2021-01-18 05:22:34,27.0587,-101.7068,55081,4696,0,50385.0,"Coahuila, Mexico",1711.270319,8.525626
395,NaN,NaN,Colima,Mexico,2021-01-18 05:22:34,19.1223,-104.0072,8591,839,0,7752.0,"Colima, Mexico",1094.181644,9.766034
396,NaN,NaN,Durango,Mexico,2021-01-18 05:22:34,24.5593,-104.6588,27087,1724,0,25363.0,"Durango, Mexico",1449.280790,6.364677


In [62]:

start_time = timeit.default_timer()
#Casos confirmados reportados por JHU ....
jhu_confirmed_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", encoding = "utf-8")
jhu_confirmed_mexico = jhu_confirmed_time_serie[jhu_confirmed_time_serie["Country/Region"] == "Mexico"]
#Muertes reportadas por JHU ...
jhu_deaths_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",encoding="utf-8")
jhu_deaths_mexico = jhu_deaths_time_serie[jhu_deaths_time_serie["Country/Region"] == "Mexico"]
#Recuperados reportados por JHU ...
jhu_recovered_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv",encoding="utf-8")
jhu_recovered_mexico = jhu_recovered_time_serie[jhu_recovered_time_serie["Country/Region"] == "Mexico"]
elapsed = timeit.default_timer() - start_time
print(elapsed)

1.4671909999999855


In [63]:
raw_confirmed_data = jhu_confirmed_mexico.reset_index(drop=True).transpose().reset_index()
raw_deats_data = jhu_deaths_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data = jhu_recovered_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data

,index,0
0,Province/State,NaN
1,Country/Region,Mexico
2,Lat,23.6345
3,Long,-102.5528
4,1/22/20,0
...,...,...
373,1/25/21,1335876
374,1/26/21,1348660
375,1/27/21,1362525
376,1/28/21,1376073


In [64]:
#Insert a character in the ith position
def insert_str(init_str, insert_str, i):
    return init_str[:i] + insert_str + init_str[i:]

In [65]:
def insert_commas(string):
    step = 3
    size = len(string)
    quotient = size//step
    mod = size % step
    
    if mod == 0:
        commas_number = quotient - 1
        init_index = step
    else:
        commas_number = quotient
        init_index = mod 

    for i in range(commas_number):
        string = insert_str(string, ",", init_index )
        init_index = init_index + step + 1
    return string

In [66]:
#Filtrando el numero de casos y fechas de cada dataFrame ...
jhu_conf_mex_df = raw_confirmed_data["index"].iloc[4:].to_frame("fecha_confirmados").reset_index(drop = True)
jhu_conf_mex_df["confirmados"] = raw_confirmed_data[0].iloc[4:].reset_index(drop = True)
jhu_conf_mex_df["fecha_confirmados_dateTime"] = pd.to_datetime(jhu_conf_mex_df["fecha_confirmados"], format = "%m/%d/%y")

jhu_deaths_mex_df = raw_deats_data["index"].iloc[4:].to_frame("fecha_muerte").reset_index(drop = True)
jhu_deaths_mex_df["muertes"] = raw_deats_data[0].iloc[4:].reset_index(drop = True)
jhu_deaths_mex_df["fecha_muerte_dateTime"] = pd.to_datetime(jhu_deaths_mex_df["fecha_muerte"], format = "%m/%d/%y")

jhu_recovered_mex_df = raw_recovered_data["index"].iloc[4:].to_frame("fecha_recuperado").reset_index(drop = True)
jhu_recovered_mex_df["recuperados"] = raw_recovered_data[0].iloc[4:].reset_index(drop = True)
jhu_recovered_mex_df["feha_recuperados_dateTime"] = pd.to_datetime(jhu_recovered_mex_df["fecha_recuperado"], format = "%m/%d/%y")

In [67]:
# Inserting commas ...
str_conf_mex = str(jhu_conf_mex_df["confirmados"].iloc[-1])
str_deaths_mex = str(jhu_deaths_mex_df["muertes"].iloc[-1])
str_recovered_mex = str(jhu_recovered_mex_df["recuperados"].iloc[-1])
#Calling inset_commas function ...
str_conf_mex_commas = insert_commas(str_conf_mex)
str_recovered_mex_commas = insert_commas(str_recovered_mex)
str_deaths_mex_commas = insert_commas(str_deaths_mex)
print(str_conf_mex_commas)
print(str_recovered_mex_commas)
print(str_deaths_mex_commas)

1,841,893
1,390,906
156,579


In [68]:
jhu_deaths_mex_df

,fecha_muerte,muertes,fecha_muerte_dateTime
0,1/22/20,0,2020-01-22
1,1/23/20,0,2020-01-23
2,1/24/20,0,2020-01-24
3,1/25/20,0,2020-01-25
4,1/26/20,0,2020-01-26
...,...,...,...
369,1/25/21,150273,2021-01-25
370,1/26/21,152016,2021-01-26
371,1/27/21,153639,2021-01-27
372,1/28/21,155145,2021-01-28


In [69]:
#Graficando los casos confirmados de muertes y confirmados ....
acum_plot(jhu_conf_mex_df["fecha_confirmados_dateTime"], jhu_conf_mex_df["confirmados"],
            jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes"], region="México")

In [70]:
# difference JHU data in order to get daily cases ....
def diff(data):
    v_1 = np.array(data)
    v_2 = np.diff(v_1)
    res = np.insert(v_2,0,v_1[0])

    return pd.Series(res)

In [71]:
#Getting daily cases ...
jhu_conf_daily = diff(jhu_conf_mex_df["confirmados"])   
jhu_conf_mex_df["casos diarios"] = jhu_conf_daily

#Getting daily deaths...
jhu_daily_deaths = diff(jhu_deaths_mex_df["muertes"]) 
jhu_deaths_mex_df["muertes diarias"] = jhu_daily_deaths

jhu_deaths_mex_df

,fecha_muerte,muertes,fecha_muerte_dateTime,muertes diarias
0,1/22/20,0,2020-01-22,0
1,1/23/20,0,2020-01-23,0
2,1/24/20,0,2020-01-24,0
3,1/25/20,0,2020-01-25,0
4,1/26/20,0,2020-01-26,0
...,...,...,...,...
369,1/25/21,150273,2021-01-25,659
370,1/26/21,152016,2021-01-26,1743
371,1/27/21,153639,2021-01-27,1623
372,1/28/21,155145,2021-01-28,1506


In [72]:
jhu_conf_mex_df

,fecha_confirmados,confirmados,fecha_confirmados_dateTime,casos diarios
0,1/22/20,0,2020-01-22,0
1,1/23/20,0,2020-01-23,0
2,1/24/20,0,2020-01-24,0
3,1/25/20,0,2020-01-25,0
4,1/26/20,0,2020-01-26,0
...,...,...,...,...
369,1/25/21,1771740,2021-01-25,8521
370,1/26/21,1788905,2021-01-26,17165
371,1/27/21,1806849,2021-01-27,17944
372,1/28/21,1825519,2021-01-28,18670


In [73]:
 
 show_confirmed_cases(jhu_conf_mex_df["fecha_confirmados_dateTime"],jhu_conf_mex_df["casos diarios"], region = "México", anomalia = True)
 show_deaths(jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes diarias"],  region = "México", anomalia = True)